In [ ]:
% run 0-utils.ipynb

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler
from tensorboardX import SummaryWriter
import skimage.transform as transform
import skimage.filters as filters
import networkx as nx
from scipy import spatial
import scipy.sparse.csgraph
from gcnn.datasets import load_icebergs
from gcnn.graph import *
from gcnn.coarsening import graclus

In [ ]:
icebergs = load_icebergs('train')

In [ ]:
icebergs.head()

In [ ]:
gr_size = 38
gr = grid_coordinates(gr_size)
gr.shape

In [ ]:
mask = knn(gr, k=6, metric='cityblock')
mask = nx.adjacency_matrix(mask, sorted(mask.nodes))

In [ ]:
from gcnn.coarsening import graclus

In [ ]:
graphs, perm = graclus.coarsen(sp.sparse.csr.csr_matrix(mask), levels=3, self_connections=False)

In [ ]:
len(perm)

In [ ]:
laps = [sp.sparse.csgraph.laplacian(g, normed=True) for g in graphs[:-1]]

In [ ]:
n = laps[0].shape[0]
n

In [ ]:
icebergs = icebergs.assign(
    graph=icebergs
        .band_1
        .apply(lambda x: transform.rescale(x.reshape(75, 75), 0.5, mode='constant'))
        .apply(lambda x: filters.gaussian(x, 2))
        .apply(lambda x: np.r_[x[tuple(gr.reshape(2, -1))], np.zeros(n - gr_size * gr_size)][perm])
)

In [ ]:
train, test = train_test_split(icebergs, test_size=0.15, stratify=icebergs.is_iceberg)
np.unique(train.is_iceberg, return_counts=True)[1] / len(train)

In [ ]:
np.unique(test.is_iceberg, return_counts=True)[1] / len(test)

In [ ]:
scaler_1 = StandardScaler()
scaler_2 = StandardScaler()
scaler_graph = StandardScaler()

In [ ]:
for _, i in train.iterrows():
    scaler_1.partial_fit(i.band_1.reshape(1, -1))
    scaler_2.partial_fit(i.band_2.reshape(1, -1))
    scaler_graph.partial_fit(i.graph.reshape(1, -1))

In [ ]:
train_x = torch.from_numpy(scaler_graph.transform(np.stack(train.graph))).float()
test_x = torch.from_numpy(scaler_graph.transform(np.stack(test.graph))).float()
train_y = torch.from_numpy(train.is_iceberg.values.reshape(-1, 1)).float()
test_y = torch.from_numpy(test.is_iceberg.values.reshape(-1, 1)).float()

In [ ]:
def fourier(laplacian):
    eigenvalues, eigenvectors = sp.linalg.eigh(laplacian)
    return eigenvectors

In [ ]:
f0 = torch.from_numpy(fourier(laps[0].todense())).float()
f0.size()

In [ ]:
f1 = torch.from_numpy(fourier(laps[1].todense())).float()
f1.size()

In [ ]:
f2 = torch.from_numpy(fourier(laps[2].todense())).float()
f2.size()

In [ ]:
cuda = torch.cuda.is_available()
num_epochs = 250
batch_size = 64
learning_rate = 0.000001
cuda

In [ ]:
train_loader = data_utils.DataLoader(data_utils.TensorDataset(train_x, train_y), batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(data_utils.TensorDataset(test_x, test_y), batch_size=batch_size, shuffle=True)

In [ ]:
class GraphConv(nn.Module):
    def __init__(self, fourier, n_filter):
        super(GraphConv, self).__init__()
        
        self.n = len(fourier)
        self.u = Variable(fourier, requires_grad=False)
        self.n_filter = n_filter
        
        self.weight = nn.Parameter(torch.Tensor(self.n, self.n_filter))
        self.bias = nn.Parameter(torch.Tensor(self.n))
        
        stdv = 1. / np.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x): # samples x n
        # fourier
        out = x @ self.u # samples x n
        
        # filter
        w = self.weight.unsqueeze(2) # n x f x 1
        out = out.t().unsqueeze(1) # n x 1 x samples
        out = w @ out # n x f x samples
        
        # un-fourier
        out = out.permute(2, 1, 0).contiguous() # samples x f x n
        out = out.view(-1, self.n) # (samples * f) x n
        out = out @ self.u.t() # (samples * f) x n
        out = out.view(-1, self.n_filter, self.n) # samples x f x n
        
        # bias?
        out = out + self.bias # samples x f x n
        return out

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)        

In [ ]:
class GraphFourierConv(nn.Module):
    def __init__(self, fourier_basis, in_channels, out_channels, bias=True):
        super().__init__()
 
        self.n = fourier_basis.size(0)
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        assert fourier_basis.size(1) == self.n
        self.u = Variable(fourier_basis, requires_grad=False) # n x n
        self.ut = self.u.t()

        self.weight = nn.Parameter(torch.Tensor(self.n, self.out_channels, 1)) # n x out x 1
        
        if bias:
            self.bias = nn.Parameter(torch.Tensor(self.out_channels, 1))
        else:
            self.register_parameter('bias', None)
            
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1. / np.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        # batch x in x n

        # fourier
        out = x.view(-1, self.n) # (batch * in) x n
        out = out @ self.u # (batch * in) x n
        
        # filter
        out = out.t().unsqueeze(1) # n x 1 x (batch * in)
        out = self.weight @ out # n x out x (batch * in)

        # un-fourier
        out = out.transpose(0, 2).contiguous() # (batch * in) x out x n
        out = out.view(-1, self.n) # (batch * in * out) x n
        out = out @ self.ut # (batch * in * out) x n
        out = out.view(-1, self.in_channels, self.out_channels, self.n) # batch x in x out x n
        
        # sum in dim + bias
        out = out.sum(1) # batch x out x n
        if self.bias is not None:
            out = out + self.bias # batch x out x n
        
        return out
    
    def __repr__(self):
        return '{}(fourier_basis, {}, {}, bias={})'.format(__class__.__name__, self.in_channels, self.out_channels, self.bias)

In [ ]:
conv1_dim = 16
conv2_dim = 16

net = nn.Sequential(
    #GraphConv(f.cuda() if cuda else f, conv1_dim),
    GraphFourierConv(f0.cuda() if cuda else f, 1, conv1_dim, bias=False),
    nn.BatchNorm2d(conv1_dim),
    nn.ReLU(),
    nn.MaxPool1d(2),
    
    GraphFourierConv(f1.cuda() if cuda else f, conv1_dim, conv2_dim, bias=False),
    nn.BatchNorm2d(conv2_dim),
    nn.ReLU(),
    nn.MaxPool1d(2),
    
    Flatten(),
    nn.Linear(len(f1) // 2 * conv2_dim, 600),
    nn.ReLU(),
    nn.Linear(600, 200),
    nn.ReLU(),
    nn.Linear(200, 1),
)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
writer = SummaryWriter()

if cuda:
    net.cuda()
    criterion.cuda()

writer.add_graph(net, net(Variable(train_x.narrow(0, 0, 1)).cuda()))
    
epoch_train_loss = []
epoch_test_loss = []

In [ ]:
def loop(loader, training=False):
    
    running_loss = 0
    targets = []
    predictions = []
    
    for batch_id, (x, y) in enumerate(loader):
        x = Variable(x)
        y = Variable(y)
        
        if cuda:
            x = x.cuda()
            y = y.cuda()
        
        optimizer.zero_grad()
        
        outputs = net(x)
        loss = criterion(outputs, y)

        if training:
            loss.backward()
            optimizer.step()

        running_loss += loss.data.cpu()[0]
        targets.extend(y.data.cpu().numpy())
        predictions.extend(outputs.sigmoid().data.cpu().numpy())
        
    return np.array(targets), np.array(predictions), running_loss

In [ ]:
for e in range(num_epochs):

    train_targets, train_preds, train_loss = loop(train_loader, training=True)
    writer.add_scalar('data/train_loss', train_loss, e)
    writer.add_scalar('data/train_accuracy', accuracy_score(train_targets, train_preds.round()), e)
      
    test_targets, test_preds, test_loss = loop(test_loader, training=False)
    writer.add_scalar('data/test_loss', test_loss, e)
    writer.add_scalar('data/test_accuracy', accuracy_score(test_targets, test_preds.round()), e)

    train_loss /= train.shape[0]
    test_loss /= test.shape[0]
    
    epoch_train_loss.append(train_loss)
    epoch_test_loss.append(test_loss)
    
    print('epoch {:3d} - Training loss: {:.4f} Testing  loss: {:.4f}'.format(e,train_loss, test_loss))

In [ ]:
plt.plot(epoch_train_loss, label='train')
plt.plot(epoch_test_loss, label='test')
plt.legend();

In [ ]:
train_targets, train_preds, train_loss = loop(train_loader)
accuracy_score(train_targets, train_preds.round())

In [ ]:
log_loss(train_targets, train_preds)

In [ ]:
test_targets, test_preds, test_loss = loop(test_loader)
accuracy_score(test_targets, test_preds.round())

In [ ]:
log_loss(test_targets, test_preds)

In [ ]:
print(net)